In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import dask
import dask.dataframe as dd
#import numba

In [2]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_new_building_merge_land_specs_20211228.csv', dtype={1:'str', 9:'Int64', 10:'Int64'})
main_data_df['계약날짜'] = pd.to_datetime(main_data_df['계약날짜'])
main_data_df = main_data_df.sort_values(['시군구', '본번', '부번', '대장구분명', '건축년도', '부동산유형', '계약날짜'])
print(main_data_df.shape)
main_data_df.head()

(5006836, 33)
Wall time: 25.4 s


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,토지일련번호,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,데이터기준일자
0,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,33.18,15900,0,1,1992,개포로109길 9,아파트,14000~16000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-03,2,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
1,서울특별시 강남구 개포동,12,12,0,삼익대청아파트,매매,39.53,24800,0,5,1992,개포로109길 21,아파트,24000~26000,삼익대청,기타,일반,대,206607.181143,443586.061143,2006-01-07,6,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
2,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,39.53,24700,0,12,1992,개포로109길 9,아파트,24000~26000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-09,8,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
3,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,49.86,33300,0,11,1992,개포로109길 9,아파트,32000~34000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-09,8,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
4,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,39.53,24700,0,4,1992,개포로109길 9,아파트,24000~26000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-10,9,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29


In [3]:
main_data_df['지번주소'] = main_data_df['시군구'] + ' ' + main_data_df['지번'] + ' ' + main_data_df['건축년도'].astype('str') + ' ' + main_data_df['부동산유형'] + ' ' + main_data_df['거래유형']

In [4]:
main_data_df['distance_x'] = np.nan
main_data_df['distance_y'] = np.nan
main_data_df['distance'] = np.nan

In [5]:
main_data_df = dd.from_pandas(main_data_df, npartitions=1).reset_index(drop=True)

In [6]:
df = main_data_df.copy()

In [7]:
def year_circular_sine_func(x):
    return math.sin(2*math.pi*((x)/365))

def year_circular_cosine_func(x):
    return math.cos(2*math.pi*((x)/365))

In [8]:
ranges_list = [1000]

numerical_cols = [
    '전용면적(㎡)', '거래/보증금(만원)', '월세(만원)', '층', '건축년도', 'x좌표', 'y좌표', '계약날짜', '계약날짜일수_2006년기준',
    '토지면적'
]

numerical_description_features_list = [
    'mean', 'std', 'min', 'quantile(0.25)', 'quantile(0.5)', 'quantile(0.75)', 'max', 'sum', 'sem', 'skew', 'var'
]

date_offset_months = 12

In [9]:
all_numerical_feature_cols = []
for target_range in ranges_list:
    for col in numerical_cols:
        for f in numerical_description_features_list:
            col_name = '최근{}개월_'.format(date_offset_months) + str(target_range) + '미터이내_' + col + '_' + f
            main_data_df[col_name] = np.nan
            all_numerical_feature_cols.append(col_name)

In [10]:
print(len(all_numerical_feature_cols))
all_numerical_feature_cols[:10]

110


['최근12개월_1000미터이내_전용면적(㎡)_mean',
 '최근12개월_1000미터이내_전용면적(㎡)_std',
 '최근12개월_1000미터이내_전용면적(㎡)_min',
 '최근12개월_1000미터이내_전용면적(㎡)_quantile(0.25)',
 '최근12개월_1000미터이내_전용면적(㎡)_quantile(0.5)',
 '최근12개월_1000미터이내_전용면적(㎡)_quantile(0.75)',
 '최근12개월_1000미터이내_전용면적(㎡)_max',
 '최근12개월_1000미터이내_전용면적(㎡)_sum',
 '최근12개월_1000미터이내_전용면적(㎡)_sem',
 '최근12개월_1000미터이내_전용면적(㎡)_skew']

In [11]:
def numerical_description_feature_creator(x, y):

    if y == 'mean':
        return x.mean()
    elif y == 'std':
        return x.std()
    elif y == 'min':
        return x.min()
    elif y == 'quantile(0.25)':
        return x.quantile(0.25)
    elif y == 'quantile(0.5)':
        return x.quantile(0.5)
    elif y == 'quantile(0.75)':
        return x.quantile(0.75)
    elif y == 'max':
        return x.max()
    elif y == 'sum':
        return x.sum()
    elif y == 'sem':
        return x.sem()
    elif y == 'skew':
        return x.skew()
    elif y == 'var':
        return x.var()
    else:
        raise ValueError('feature name error')

In [12]:
def groupby_function(x):
    # x is a groupby dataframe

    # vars
    coor_x = x['x좌표'].iloc[0]
    coor_y = x['y좌표'].iloc[0]
    addr = x['지번주소'].iloc[0]
    house_type = x['지번주소'].iloc[0].split(' ')[-2]
    trade_type = x['지번주소'].iloc[0].split(' ')[-1]
    built_year = int(x['건축년도'].iloc[0])
    unique_contract_dates = x['계약날짜'].unique()

    #
    df['distance_x'] = df['x좌표'] - coor_x
    df['distance_y'] = df['y좌표'] - coor_y
    df['distance'] = np.sqrt(df['distance_x']**2 + df['distance_y']**2)

    for target_range in ranges_list:

        range_df = df[
            (df['distance'] <= target_range)
            &(df['지번주소'] != addr)
            &(df['부동산유형'] == house_type)
            &(df['거래유형'] == trade_type)
            &(df['건축년도'] <= built_year)
            &(df['건축년도'] > (built_year - 5))
        ]

        #
        if range_df.shape[0] == 0:
            target_range_cols = [col for col in all_numerical_feature_cols if str(target_range) in col]

            i_list = []
            for i in range(x.shape[0]):
                j_list = []
                for j in range(len(target_range_cols)):
                    j_list.append(np.nan)

                i_list.append(j_list)

            nan_df = pd.DataFrame(np.array(i_list), columns=target_range_cols)

            return nan_df

        #
        else:

            for contract_date in unique_contract_dates:
                contract_date = pd.to_datetime(contract_date)

                original_df = x[x['계약날짜'] == contract_date]

                contract_date_df = range_df[
                    (range_df['계약날짜'] <= contract_date)
                    &(range_df['계약날짜'] > (contract_date - pd.DateOffset(months=date_offset_months)))
                ]

                if contract_date_df.shape[0] == 0:
                    target_range_cols = [col for col in all_numerical_feature_cols if str(target_range) in col]

                    i_list = []
                    for i in range(original_df.shape[0]):
                        j_list = []
                        for j in range(len(target_range_cols)):
                            j_list.append(np.nan)

                        i_list.append(j_list)

                    nan_df = pd.DataFrame(np.array(i_list), columns=target_range_cols)

                    x.loc[original_df.index, target_range_cols] = nan_df
                    #return nan_df


                #
                else:


                    for num_col in numerical_cols:

                        for num_feat in numerical_description_features_list:

                            feat_col_name = '최근{}개월_'.format(date_offset_months) + str(target_range) + '미터이내_' + num_col + '_' + num_feat

                            try:
                                x.loc[original_df.index, feat_col_name] = numerical_description_feature_creator(contract_date_df[num_col], num_feat)
                            except:
                                x.loc[original_df.index, feat_col_name] = np.nan

    return x[all_numerical_feature_cols]

In [13]:
meta_df = main_data_df.head(0)
print(meta_df.shape)
meta_df.head()

(0, 146)


c:\users\max6296\ml\lib\site-packages\dask\dataframe\methods.py:333: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = val


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,토지일련번호,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,데이터기준일자,distance_x,distance_y,distance,최근12개월_1000미터이내_전용면적(㎡)_mean,최근12개월_1000미터이내_전용면적(㎡)_std,최근12개월_1000미터이내_전용면적(㎡)_min,최근12개월_1000미터이내_전용면적(㎡)_quantile(0.25),최근12개월_1000미터이내_전용면적(㎡)_quantile(0.5),최근12개월_1000미터이내_전용면적(㎡)_quantile(0.75),최근12개월_1000미터이내_전용면적(㎡)_max,최근12개월_1000미터이내_전용면적(㎡)_sum,최근12개월_1000미터이내_전용면적(㎡)_sem,최근12개월_1000미터이내_전용면적(㎡)_skew,최근12개월_1000미터이내_전용면적(㎡)_var,최근12개월_1000미터이내_거래/보증금(만원)_mean,최근12개월_1000미터이내_거래/보증금(만원)_std,최근12개월_1000미터이내_거래/보증금(만원)_min,최근12개월_1000미터이내_거래/보증금(만원)_quantile(0.25),최근12개월_1000미터이내_거래/보증금(만원)_quantile(0.5),최근12개월_1000미터이내_거래/보증금(만원)_quantile(0.75),최근12개월_1000미터이내_거래/보증금(만원)_max,최근12개월_1000미터이내_거래/보증금(만원)_sum,최근12개월_1000미터이내_거래/보증금(만원)_sem,최근12개월_1000미터이내_거래/보증금(만원)_skew,최근12개월_1000미터이내_거래/보증금(만원)_var,최근12개월_1000미터이내_월세(만원)_mean,최근12개월_1000미터이내_월세(만원)_std,최근12개월_1000미터이내_월세(만원)_min,최근12개월_1000미터이내_월세(만원)_quantile(0.25),최근12개월_1000미터이내_월세(만원)_quantile(0.5),최근12개월_1000미터이내_월세(만원)_quantile(0.75),최근12개월_1000미터이내_월세(만원)_max,최근12개월_1000미터이내_월세(만원)_sum,최근12개월_1000미터이내_월세(만원)_sem,최근12개월_1000미터이내_월세(만원)_skew,최근12개월_1000미터이내_월세(만원)_var,최근12개월_1000미터이내_층_mean,최근12개월_1000미터이내_층_std,최근12개월_1000미터이내_층_min,최근12개월_1000미터이내_층_quantile(0.25),최근12개월_1000미터이내_층_quantile(0.5),최근12개월_1000미터이내_층_quantile(0.75),최근12개월_1000미터이내_층_max,최근12개월_1000미터이내_층_sum,최근12개월_1000미터이내_층_sem,최근12개월_1000미터이내_층_skew,최근12개월_1000미터이내_층_var,최근12개월_1000미터이내_건축년도_mean,최근12개월_1000미터이내_건축년도_std,최근12개월_1000미터이내_건축년도_min,최근12개월_1000미터이내_건축년도_quantile(0.25),최근12개월_1000미터이내_건축년도_quantile(0.5),최근12개월_1000미터이내_건축년도_quantile(0.75),최근12개월_1000미터이내_건축년도_max,최근12개월_1000미터이내_건축년도_sum,최근12개월_1000미터이내_건축년도_sem,최근12개월_1000미터이내_건축년도_skew,최근12개월_1000미터이내_건축년도_var,최근12개월_1000미터이내_x좌표_mean,최근12개월_1000미터이내_x좌표_std,최근12개월_1000미터이내_x좌표_min,최근12개월_1000미터이내_x좌표_quantile(0.25),최근12개월_1000미터이내_x좌표_quantile(0.5),최근12개월_1000미터이내_x좌표_quantile(0.75),최근12개월_1000미터이내_x좌표_max,최근12개월_1000미터이내_x좌표_sum,최근12개월_1000미터이내_x좌표_sem,최근12개월_1000미터이내_x좌표_skew,최근12개월_1000미터이내_x좌표_var,최근12개월_1000미터이내_y좌표_mean,최근12개월_1000미터이내_y좌표_std,최근12개월_1000미터이내_y좌표_min,최근12개월_1000미터이내_y좌표_quantile(0.25),최근12개월_1000미터이내_y좌표_quantile(0.5),최근12개월_1000미터이내_y좌표_quantile(0.75),최근12개월_1000미터이내_y좌표_max,최근12개월_1000미터이내_y좌표_sum,최근12개월_1000미터이내_y좌표_sem,최근12개월_1000미터이내_y좌표_skew,최근12개월_1000미터이내_y좌표_var,최근12개월_1000미터이내_계약날짜_mean,최근12개월_1000미터이내_계약날짜_std,최근12개월_1000미터이내_계약날짜_min,최근12개월_1000미터이내_계약날짜_quantile(0.25),최근12개월_1000미터이내_계약날짜_quantile(0.5),최근12개월_1000미터이내_계약날짜_quantile(0.75),최근12개월_1000미터이내_계약날짜_max,최근12개월_1000미터이내_계약날짜_sum,최근12개월_1000미터이내_계약날짜_sem,최근12개월_1000미터이내_계약날짜_skew,최근12개월_1000미터이내_계약날짜_var,최근12개월_1000미터이내_계약날짜일수_2006년기준_mean,최근12개월_1000미터이내_계약날짜일수_2006년기준_std,최근12개월_1000미터이내_계약날짜일수_2006년기준_min,최근12개월_1000미터이내_계약날짜일수_2006년기준_quantile(0.25),최근12개월_1000미터이내_계약날짜일수_2006년기준_quantile(0.5),최근12개월_1000미터이내_계약날짜일수_2006년기준_quantile(0.75),최근12개월_1000미터이내_계약날짜일수_2006년기준_max,최근12개월_1000미터이내_계약날짜일수_2006년기준_sum,최근12개월_1000미터이내_계약날짜일수_2006년기준_sem,최근12개월_1000미터이내_계약날짜일수_2006년기준_skew,최근12개월_1000미터이내_계약날짜일수_2006년기준_var,최근12개월_1000미터이내_토지면적_mean,최근12개월_1000미터이내_토지면적_std,최근12개월_1000미터이내_토지면적_min,최근12개월_1000미터이내_토지면적_quantile(0.25),최근12개월_1000미터이내_토지면적_quantile(0.5),최근12개월_1000미터이내_토지면적_quantile(0.75),최근12개월_1000미터이내_토지면적_max,최근12개월_1000미터이내_토지면적_sum,최근12개월_1000미터이내_토지면적_sem,최근12개월_1000미터이내_토지면적_skew,최근12개월_1000미터이내_토지면적_var


In [14]:
meta_df.dtypes

시군구                                                      object
지번                                                       object
본번                                                        int64
부번                                                        int64
건물/단지명                                                   object
거래유형                                                     object
전용면적(㎡)                                                 float64
거래/보증금(만원)                                                int64
월세(만원)                                                    int64
층                                                         Int64
건축년도                                                      Int64
도로명                                                      object
부동산유형                                                    object
가격범위                                                     object
건물단지명_키워드                                                object
브랜드                                     

In [15]:
%%time
main_data_df[all_numerical_feature_cols] = main_data_df.groupby(['시군구', '지번', '건축년도', '부동산유형', '거래유형']).apply(groupby_function, meta=meta_df).compute()

c:\users\max6296\ml\lib\site-packages\dask\dataframe\methods.py:333: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = val


KeyError: '건축년도'

In [16]:
%%time
main_data_df.to_csv('./Prepped Data/range_data_20220112.csv', index=False)

NameError: name 'main_data_ddf' is not defined

In [ ]:
@numba.jit
def temp_func(df):
    import pandas as pd
    import numpy as np
    
    df = df.copy()
    
    ranges_list = [1000]
    
    numerical_cols = ['전용면적(㎡)', '거래/보증금(만원)', '월세(만원)', '층', '건축년도', 'x좌표', 'y좌표', '계약날짜', '계약날짜일수_2006년기준', '토지면적']
    
    numerical_description_features_list = [
        'mean', 'std', 'min', 'quantile(0.25)', 'quantile(0.5)', 'quantile(0.75)', 'max', 'sum', 'sem', 'skew', 'var'
    ]
    
    date_offset_months = 12    
    
    all_numerical_feature_cols = []
    for target_range in ranges_list:
        for col in numerical_cols:
            for f in numerical_description_features_list:
                col_name = '최근{}개월_'.format(date_offset_months) + str(target_range) + '미터이내_' + col + '_' + f
                df[col_name] = np.nan
                all_numerical_feature_cols.append(col_name)
    
    def numerical_description_feature_creator(x, y):
    
        if y == 'mean':
            return x.mean()
        elif y == 'std':
            return x.std()
        elif y == 'min':
            return x.min()
        elif y == 'quantile(0.25)':
            return x.quantile(0.25)
        elif y == 'quantile(0.5)':
            return x.quantile(0.5)
        elif y == 'quantile(0.75)':
            return x.quantile(0.75)
        elif y == 'max':
            return x.max()
        elif y == 'sum':
            return x.sum()
        elif y == 'sem':
            return x.sem()
        elif y == 'skew':
            return x.skew()
        elif y == 'var':
            return x.var()
        else:
            raise ValueError('feature name error')
     
    def groupby_function(x):
        # x is a groupby dataframe

        # vars
        coor_x = x['x좌표'].iloc[0]
        coor_y = x['y좌표'].iloc[0]
        addr = x['지번주소'].iloc[0]
        house_type = x['지번주소'].iloc[0].split(' ')[-2]
        trade_type = x['지번주소'].iloc[0].split(' ')[-1]
        print(trade_type)
        built_year = int(x['건축년도'].iloc[0])
        print(built_year)
        unique_contract_dates = x['계약날짜'].unique()

        #
        df['distance_x'] = df['x좌표'] - coor_x
        df['distance_y'] = df['y좌표'] - coor_y
        df['distance'] = np.sqrt(df['distance_x']**2 + df['distance_y']**2)
        
        for target_range in ranges_list:

            range_df = df[
                (df['distance'] <= target_range)
                &(df['지번주소'] != addr)
                &(df['부동산유형'] == house_type)
                &(df['거래유형'] == trade_type)
                &(df['건축년도'] <= built_year)
                &(df['건축년도'] > (built_year - 5))
            ]
            
            #
            if range_df.shape[0] == 0:
                target_range_cols = [col for col in all_numerical_feature_cols if str(target_range) in col]
                
                i_list = []
                for i in range(x.shape[0]):
                    j_list = []
                    for j in range(len(target_range_cols)):
                        j_list.append(np.nan)
                    
                    i_list.append(j_list)
                
                nan_df = pd.DataFrame(np.array(i_list), columns=target_range_cols)
                
                return nan_df
            
            #
            else:
                
                for contract_date in unique_contract_dates:
                    contract_date = pd.to_datetime(contract_date)
                    
                    original_df = x[x['계약날짜'] == contract_date]
                    
                    contract_date_df = range_df[
                        (range_df['계약날짜'] <= contract_date)
                        &(range_df['계약날짜'] > (contract_date - pd.DateOffset(months=date_offset_months)))
                    ]
                    
                    if contract_date_df.shape[0] == 0:
                        target_range_cols = [col for col in all_numerical_feature_cols if str(target_range) in col]
                        
                        i_list = []
                        for i in range(original_df.shape[0]):
                            j_list = []
                            for j in range(len(target_range_cols)):
                                j_list.append(np.nan)

                            i_list.append(j_list)

                        nan_df = pd.DataFrame(np.array(i_list), columns=target_range_cols)
                        
                        x.loc[original_df.index, target_range_cols] = nan_df
                        #return nan_df
                    
                    
                    #
                    else:
                        
                        
                        for num_col in numerical_cols:
                            
                            for num_feat in numerical_description_features_list:
                                
                                feat_col_name = '최근{}개월_'.format(date_offset_months) + str(target_range) + '미터이내_' + num_col + '_' + num_feat
                                
                                try:
                                    x.loc[original_df.index, feat_col_name] = numerical_description_feature_creator(contract_date_df[num_col], num_feat)
                                except:
                                    x.loc[original_df.index, feat_col_name] = np.nan
                        
        return x[all_numerical_feature_cols]
    
    df[all_numerical_feature_cols] = df.groupby(['시군구', '지번', '건축년도', '부동산유형', '거래유형']).apply(groupby_function).reset_index(drop=True)
    
    return df

In [ ]:
sample_df = main_data_df.sample(frac=0.01)
print(sample_df.shape)
sample_df.head()

In [ ]:
prac_df = temp_func(sample_df)
print(prac_df.shape)
prac_df.head()